In [106]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

import chembl_webresource_client
from chembl_webresource_client.new_client import new_client
import rdkit
from rdkit import Chem
from rdkit.Chem import Descriptors, Lipinski

from rdkit import Chem
from rdkit.Chem.SaltRemover import SaltRemover

# Get Data

In [60]:
def getData(keyword,target_row):
    # Search the target 'entamoeba'
    target = new_client.target
    target_query = target.search(keyword)
    targets = pd.DataFrame.from_dict(target_query)

    # Select the target's chembl_id
    selected_target = targets.target_chembl_id[target_row]

    # Search for the activity based on the chembl_id
    activity = new_client.activity
    res = activity.filter(target_chembl_id = selected_target).filter(standard_type="IC50")
    act_data = pd.DataFrame(res)

    return act_data

act_data = getData('entamoeba', 0)
act_data

,activity_comment,activity_id,activity_properties,assay_chembl_id,assay_description,assay_type,assay_variant_accession,assay_variant_mutation,bao_endpoint,bao_format,...,target_organism,target_pref_name,target_tax_id,text_value,toid,type,units,uo_units,upper_value,value
0,None,145127,[],CHEMBL676675,In vitro antiprotozoal activity against Entamo...,F,None,None,BAO_0000190,BAO_0000218,...,Entamoeba histolytica,Entamoeba histolytica,5759,None,None,IC50,uM,UO_0000065,None,0.069
1,None,149779,[],CHEMBL676675,In vitro antiprotozoal activity against Entamo...,F,None,None,BAO_0000190,BAO_0000218,...,Entamoeba histolytica,Entamoeba histolytica,5759,None,None,IC50,uM,UO_0000065,None,0.022
2,None,150988,[],CHEMBL676675,In vitro antiprotozoal activity against Entamo...,F,None,None,BAO_0000190,BAO_0000218,...,Entamoeba histolytica,Entamoeba histolytica,5759,None,None,IC50,uM,UO_0000065,None,0.35
3,None,153478,[],CHEMBL676675,In vitro antiprotozoal activity against Entamo...,F,None,None,BAO_0000190,BAO_0000218,...,Entamoeba histolytica,Entamoeba histolytica,5759,None,None,IC50,uM,UO_0000065,None,0.046
4,None,157179,[],CHEMBL676675,In vitro antiprotozoal activity against Entamo...,F,None,None,BAO_0000190,BAO_0000218,...,Entamoeba histolytica,Entamoeba histolytica,5759,None,None,IC50,uM,UO_0000065,None,0.04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1667,None,23172379,"[{'comments': None, 'relation': '=', 'result_f...",CHEMBL4814683,Antiamoebic activity against Entamoeba histoly...,F,None,None,BAO_0000190,BAO_0000218,...,Entamoeba histolytica,Entamoeba histolytica,5759,None,None,IC50,uM,UO_0000065,None,0.26
1668,None,23172380,"[{'comments': None, 'relation': '=', 'result_f...",CHEMBL4814683,Antiamoebic activity against Entamoeba histoly...,F,None,None,BAO_0000190,BAO_0000218,...,Entamoeba histolytica,Entamoeba histolytica,5759,None,None,IC50,uM,UO_0000065,None,0.26
1669,None,23172381,"[{'comments': None, 'relation': '=', 'result_f...",CHEMBL4814683,Antiamoebic activity against Entamoeba histoly...,F,None,None,BAO_0000190,BAO_0000218,...,Entamoeba histolytica,Entamoeba histolytica,5759,None,None,IC50,uM,UO_0000065,None,0.15
1670,None,23172398,"[{'comments': None, 'relation': '=', 'result_f...",CHEMBL4814689,Antiamoebic activity against Entamoeba histoly...,F,None,None,BAO_0000190,BAO_0000218,...,Entamoeba histolytica,Entamoeba histolytica,5759,None,None,IC50,uM,UO_0000065,None,5.0


## Exploring Data with Lipinski descriptors
Christopher Lipinski, a scientist at Pfizer, came up with a set of rule-of-thumb for evaluating the druglikeness of compounds. Such druglikeness is based on the Absorption, Distribution, Metabolism and Excretion (ADME) that is also known as the pharmacokinetic profile. Lipinski analyzed all orally active FDA-approved drugs in the formulation of what is to be known as the Rule-of-Five or Lipinski's Rule.

The Lipinski's Rule stated the following:

* Molecular weight < 500 Dalton
* Octanol-water partition coefficient (LogP) < 5
* Hydrogen bond donors < 5
* Hydrogen bond acceptors < 10

## Convert IC50 to pIC50

To allow **IC50** data to be more uniformly distributed, we will convert **IC50** to the negative logarithmic scale which is essentially $-log10(IC50)$

This custom function **pIC50()** will accept a DataFrame as input and will:

* Take the **IC50** values from the standard_value column and converts it from nM to M by multiplying the value by $10^{-9}$
* Take the molar value and apply $-log10$
* Delete the **standard_value** column and create a new *pIC50* column

In [219]:
def dataProcessing(data):

    # Lipinski function
    # Inspired by: https://codeocean.com/explore/capsules?query=tag:data-curation
    def lipinski(smiles, verbose=False):

        moldata= []
        for elem in smiles:
            mol=Chem.MolFromSmiles(elem) 
            moldata.append(mol)
        
        baseData= np.arange(1,1)
        i=0  
        for mol in moldata:        
            desc_MolWt = Descriptors.MolWt(mol)
            desc_MolLogP = Descriptors.MolLogP(mol)
            desc_NumHDonors = Lipinski.NumHDonors(mol)
            desc_NumHAcceptors = Lipinski.NumHAcceptors(mol)
                
            row = np.array([desc_MolWt,
                            desc_MolLogP,
                            desc_NumHDonors,
                            desc_NumHAcceptors])   
        
            if(i==0):
                baseData=row
            else:
                baseData=np.vstack([baseData, row])
            i=i+1      
        
        columnNames=["MW","LogP","NumHDonors","NumHAcceptors"]   
        descriptors = pd.DataFrame(data=baseData,columns=columnNames)
        
        return descriptors

    # Drop the null values from standard_value and canonical_smiles
    new_data = data
    new_data = new_data[new_data['standard_value'].notna()]
    new_data = new_data[new_data['canonical_smiles'].notna()]

    # Reset Index
    new_data.reset_index(inplace=True)

    # Add the bioactivity classification
    bioactivity_class =[]
    for i in new_data['standard_value']:
        if float(i) <= 1000:
            bioactivity_class.append('active')
        elif float(i) >= 10000:
            bioactivity_class.append('inactive')
        else:
            bioactivity_class.append('intermidiate')

    # Grab the molecule_chembl_id, canonical_smiles, standard_value
    # get molecule_chembl_id, canonical_smiles, standard_value
    data_2 = new_data[['molecule_chembl_id', 'canonical_smiles', 'standard_value']]
    data_2.insert(2, 'bioactivity_class', bioactivity_class)

    # run lipinski function
    df_lipinski = lipinski(data_2['canonical_smiles'])

    # Combine the lipinski result with the rest of the data
    df_combined = pd.concat([data_2, df_lipinski], axis=1)

    # change standard_value format to float
    df_combined['standard_value'] = df_combined['standard_value'].astype('float')

    # Convert IC50 to pIC50
    # https://github.com/chaninlab/estrogen-receptor-alpha-qsar/blob/master/02_ER_alpha_RO5.ipynb

    def pIC50(input):
        pIC50 = []

        for i in input['standard_value']:
            molar = i*(10**-9) # Converts nM to M
            pIC50.append(-np.log10(molar))

        input['pIC50'] = pIC50
        x = input.drop('standard_value', 1)
            
        return x
    
    # find the pIC50 value
    new_df = pIC50(df_combined)

    #drop the negative
    positive_df = new_df[new_df['bioactivity_class'] != 'intermidiate']

    # reset index
    positive_df.reset_index(inplace=True)
    positive_df.drop(columns='index', axis=1, inplace=True)

    # Remove Salt
    remover = SaltRemover() # use default saltremover
    mol_list = []

    for i in positive_df.canonical_smiles:
        mol = Chem.MolFromSmiles(i)
        stripped = remover.StripMol(mol)
        stereo = Chem.RemoveStereochemistry(stripped) 
        mol_list.append(Chem.MolToSmiles(stripped))

    # replace old canonical_smiles with the new one
    positive_df['canonical_smiles'] = pd.Series(mol_list)

    return positive_df

In [221]:
positive_df = dataProcessing(act_data)
positive_df

# export to csv
# final_data.to_csv('training_entamoeba_canonicalSmiles.csv', index=False)
# final_data.to_csv('testing_entamoeba_canonicalSmiles.csv', index=False)
# final_data.to_csv('entamoeba_canonicalSmiles.csv', index=False)

/var/folders/1t/8qrjy_l91msg5bslbmh7qdp40000gn/T/ipykernel_1192/3539744201.py:79: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  x = input.drop('standard_value', 1)
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/pandas/core/frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(
/var/folders/1t/8qrjy_l91msg5bslbmh7qdp40000gn/T/ipykernel_1192/3539744201.py:104: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  positive_df['ca

,molecule_chembl_id,canonical_smiles,bioactivity_class,MW,LogP,NumHDonors,NumHAcceptors,pIC50
0,CHEMBL55641,FC(F)(F)c1nc2ccccc2[nH]1,active,186.136,2.58170,1.0,1.0,7.161151
1,CHEMBL53788,FC(F)(F)c1nc2cc(Cl)ccc2[nH]1,active,220.581,3.23510,1.0,1.0,7.657577
2,CHEMBL137,Cc1ncc([N+](=O)[O-])n1CCO,active,171.156,0.09202,1.0,5.0,6.455932
3,CHEMBL56473,Cn1c(C(F)(F)F)nc2cc(Cl)ccc21,active,234.608,3.24550,0.0,2.0,7.337242
4,CHEMBL293520,Cn1c(C(F)(F)F)nc2ccccc21,active,200.163,2.59210,0.0,2.0,7.397940
...,...,...,...,...,...,...,...,...
1112,CHEMBL4856806,Cc1ncc([N+](=O)[O-])n1CCOC(=O)Nc1ccc(OS(=O)(=O...,active,514.438,4.13492,1.0,9.0,6.619789
1113,CHEMBL3960281,Cc1ncc([N+](=O)[O-])n1CCOC(=O)Nc1ccc(OCc2ccc(F...,active,414.393,4.06652,1.0,7.0,7.096910
1114,CHEMBL3932812,Cc1ncc([N+](=O)[O-])n1CCOC(=O)Nc1ccc(OCc2ccc(F...,active,432.383,4.20562,1.0,7.0,6.585027
1115,CHEMBL4855961,Cc1ncc([N+](=O)[O-])n1CCOC(=O)Nc1ccc(OCc2cccc(...,active,522.299,4.53202,1.0,7.0,6.585027


# Part 3 - Encoding

note: 
* Canonical_smiles: represents chemical information that partain to the chemical structure


In [ ]:
# download padel descriptor
# ! wget https://github.com/dataprofessor/bioinformatics/raw/master/padel.zip
# ! wget https://github.com/dataprofessor/bioinformatics/raw/master/padel.sh
# ! unzip padel.zip

In [90]:
def canon_encode(data_path):
    df3 = pd.read_csv(data_path)
    df3_selection = df3[['canonical_smiles', 'molecule_chembl_id']]
    df3_selection.to_csv('molecule.smi', sep='\t', index=False, header=False)

    ! bash padel.sh

    df3_x = pd.read_csv('descriptors_output.csv')
    df3_x = df3_x.drop(columns= 'Name')

    dataset3 = pd.concat([df3_x, df3['pIC50']], axis=1)

    return dataset3